In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

Using TensorFlow backend.


In [2]:
def data():
    X_train = pd.read_csv("X_train1.txt",header=None)
    y_train = pd.read_csv("y_train1.txt",header=None)
    X_val = pd.read_csv("X_test1.txt",header=None)
    y_val = pd.read_csv("y_test1.txt",header=None)
    x_train = np.array(X_train)
    ytrain = np.array(y_train)
    x_val = np.array(X_val)
    yval = np.array(y_val)
    x_train = x_train.reshape((210, 70,6))
    ytrain = to_categorical(ytrain)
    x_val = x_val.reshape((70,70,6))
    yval = to_categorical(yval)
    return x_train,ytrain,x_val,yval

In [3]:
def model(x_train,ytrain,x_val,yval):
    model = Sequential()
    model.add(LSTM(units={{choice([20,50,70,100])}}, return_sequences = True, input_shape =(x_train.shape[1],6)))
    model.add(Dropout({{uniform(0,1)}}))
    model.add(LSTM(units={{choice([20,50,80,100])}}))
    model.add(Dropout({{uniform(0,1)}}))
    if conditional({{choice(['one','two'])}})== 'two':
        model.add(Dense({{choice([10,50,70])}},activation='relu'))
        model.add(Dropout({{uniform(0,1)}}))
        
    model.add(Dense(ytrain.shape[1],activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    
    model.fit(x_train,ytrain ,verbose = 0, batch_size={{choice([5,7,10,35])}},
              validation_data=(x_val,yval))
    
    score, acc = model.evaluate(x_val,yval,verbose=0)
    print('Test accuracy:',acc)
    return {'loss':-acc,'status':STATUS_OK,'model':model}
    

In [4]:
x_train, ytrain, x_val, yval = data()
best_run, best_model = optim.minimize(model=model,
                                     data=data,
                                     algo=tpe.suggest,
                                     max_evals=30,
                                     trials=Trials(),
                                     notebook_name ='Optimizer 2')


>>> Imports:
#coding=utf-8

try:
    import tensorflow as tf
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import LSTM, Dense, Dropout
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'units': hp.choice('units', [20,50,70,100]),
        'Dropout': hp.uniform('Dropout', 0,1),
        'units_1': hp.choice('units_1', [20,50,80,100]),
        'Dropout_1': hp.uniform('Dropout_1', 0,1),
        'conditional': hp.c

In [5]:
print(best_model.evaluate(x_val, yval))
print(best_run)

70/70 [==============================] - 0s 698us/step
[0.7264742970466613, 1.0]
{'Dense': 1, 'Dropout': 0.5966022018243913, 'Dropout_1': 0.28539853730334785, 'Dropout_2': 0.9928052921866384, 'batch_size': 1, 'conditional': 0, 'units': 2, 'units_1': 1}
